给定一个单链表，把所有的奇数节点和偶数节点分别排在一起。请注意，这里的奇数节点和偶数节点指的是节点编号的奇偶性，而不是节点的值的奇偶性。

请尝试使用原地算法完成。你的算法的空间复杂度应为 O(1)，时间复杂度应为 O(nodes)，nodes 为节点总数。

示例 1:

输入: 1->2->3->4->5->NULL

输出: 1->3->5->2->4->NULL


示例 2:

输入: 2->1->3->5->6->4->7->NULL 

输出: 2->3->6->7->1->5->4->NULL

说明:

应当保持奇数节点和偶数节点的相对顺序。

链表的第一个节点视为奇数节点，第二个节点视为偶数节点，以此类推。


In [3]:
"""
和86-Partitions List思路几乎一样
"""

# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

class Solution:
    cnt = 1

    def oddEvenList(self, head: ListNode) -> ListNode:
        dummy_odd = odd = ListNode(None)
        dummy_even = even = ListNode(None)

        while head:
            if self.cnt % 2:  # odd
                odd.next = head
                odd = odd.next
            else:  # even
                even.next = head
                even = even.next
            self.cnt += 1
            head = head.next

        even.next = None
        odd.next = dummy_even.next

        return dummy_odd.next
        
        
        
        
        

In [4]:
"""
官方题解，只需4个指针，因为odd和left指针当做原链表的迭代器指针。
not even: odd-even-odd-^
not even.next: odd-even-odd-even-^ 
"""
class Solution:
    def oddEvenList(self, head: ListNode) -> ListNode:
        if not head:
            return head
        
        odd = head
        even = even_head = odd.next
        # 需要下两个位置
        while even and even.next:
            odd.next = even.next # 需要even not None
            odd = odd.next
            even.next = odd.next # 需要even.enxt not None
            even = even.next
        # 由于每次都是一对（odd,even）向后移动，保证了even.next后面不会有odd节点
        odd.next = even_head
        return head
        


In [5]:
"""
递归写法：主要是在回溯过程中完成
"""

class Solution:
    def oddEvenList(self, head: ListNode) -> ListNode:
        dummy_head = ListNode(None)
        dummy_head.next = head

        pre = dummy_head
        cur = head

        def recursive(pre, cur, idx):
            if cur is None or cur.next is None:
                if idx % 2 == 0:  # 最后一个节点是even
                    tail = ListNode(None)  # 创造一个假的奇数节点，以保证child_gap一定存在
                    tail.next = cur
                    return tail, tail
                else:  # 最后一个节点是odd
                    return cur, cur

            child_head, child_gap = recursive(pre.next, cur.next, idx + 1)

            if idx % 2:  # odd
                if child_head.val is None:  # 如果遇到假的奇数节点，删除之
                    cur.next = child_head.next
                    child_head.next = None
                    return cur, cur
                cur.next = child_head
                return cur, child_gap
            # even, 由于base case中保证了child_gap存在
            cur.next = child_gap.next
            child_gap.next = cur
            pre.next = child_head  # 原来pre.next指向cur，消除野指针
            return child_head, child_gap

        new_head, _ = recursive(pre, cur, 1)

        return new_head

In [6]:
"""
递归写法：主要在递进过程完成，和非递归写法几乎一样。
"""

class Solution:
    def oddEvenList(self, head: ListNode) -> ListNode:
        dummy_odd = odd = ListNode(None)
        dummy_even = even = ListNode(None)

        def recursive(head, idx):
            nonlocal odd, even
            if head is None:
                return

            if idx % 2:  # odd
                odd.next = head
                odd = odd.next
            else:
                even.next = head
                even = even.next

            recursive(head.next, idx + 1)
            return

        recursive(head, 1)
        even.next = None
        odd.next = dummy_even.next
        return dummy_odd.next

In [ ]:
"""
在之前回溯解法中：
1. 在递归函数中已经将奇偶链表连起来
2. 每次子链都是仅仅减少一个元素
所以需要根据讨论当前遍历的元素是奇是偶

根据24改进的递归，
1. 每次子链都减少一对元素（2个）
2. 返回的是奇元素子链和偶元素子链的头结点
3. 多出的步骤：找到奇链表的尾部
4. 将奇偶链表拼接起来。

避免了对奇偶的讨论
"""

class Solution:
    def oddEvenList(self, head: ListNode) -> ListNode:
        
        odd_tail = None
        def recursive(head):
            nonlocal odd_tail
            if not head:
                return None, None
            if not head.next:
                return head, None

            nxt = head.next
            odd, even = recursive(nxt.next)
            
            if not odd:
                odd_tail = head
            elif not even:
                odd_tail = odd
                
            head.next = odd
            nxt.next = even
            return head, nxt
        
        a, b = recursive(head)
        if odd_tail:
            odd_tail.next = b
        return a
        
        
        